In [1]:
!nvidia-smi

Thu Apr 25 11:41:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:90:00.0 Off |                    0 |
| N/A   38C    P0              76W / 700W |      0MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# fix numpy in colab
import numpy
from IPython.display import clear_output


!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --cache-dir $TMP_DIR --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo2


/home/hice1/abambhaniya3/pace-ice-home-data/mixtral-offloading-residency-info/notebooks/Mixtral-8x7B-Instruct-v0.1-offloading-demo2


In [3]:

import os, sys
script_dir = os.getcwd()
module_path = script_dir
for _ in range(1):
    module_path = os.path.abspath(os.path.join(module_path, '../'))
    if module_path not in sys.path:
        sys.path.insert(0,module_path)
        
sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
import time
import gc
from src.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/home/hice1/abambhaniya3/scratch/moe_offload/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2  

In [5]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo2"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 6
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


# del model

gc.collect
torch.cuda.empty_cache()


## DEFAULT with 4 Experts on GPU

In [25]:
if 'model' in locals():
    del model
model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
    routing_strategy="TOP-K",
    routing_threshold=0.00
)

/home/hice1/abambhaniya3/scratch/moe_offload/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Loading experts: 100%|██████████| 32/32 [00:04<00:00,  6.99it/s]


In [26]:
from transformers import TextStreamer


tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0
# while True:
user_input = "Where is Georgia Tech? What is the name of its mascot?"

user_entry = dict(role="user", content=user_input)
input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(model.device)

if past_key_values is None:
  attention_mask = torch.ones_like(input_ids)
else:
  seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
  attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=model.device)



# sequence = result["sequences"]
# past_key_values = result["past_key_values"]

In [27]:
start_time = time.time()
result = model.generate(
  input_ids=input_ids,
  attention_mask=attention_mask,
  past_key_values=past_key_values,
  streamer=streamer,
  do_sample=True,
  temperature=0.9,
  top_p=0.9,
  min_new_tokens=100,
  max_new_tokens=100,
  pad_token_id=tokenizer.eos_token_id,
  return_dict_in_generate=True,
  output_hidden_states=False,
  output_router_logits=True,
  output_scores=True, 
)
latency = time.time() - start_time
total_experts_saved = 0
for i in result['router_logits'][-32:]:
    total_experts_saved += i[1]
total_experts_saved 

Num_tokens = result['sequences'].shape[1] - input_ids.shape[1]
print(f"Total Latency :{latency} sec, Throughput:{Num_tokens/latency}, Expert Loads saved:{total_experts_saved} ")

Georgia Tech, officially known as the Georgia Institute of Technology, is located in Atlanta, Georgia, in the southeastern United States. The school's mascot is a yellow, costumed bird known as the "Georgia Tech Ramblin' Wreck." This is also the name of the converted 1930 Ford Model A that leads the Georgia Tech football team onto the field during home games. The Ramblin' Wreck has been a part of Georgia Tech
Total Latency :26.38260054588318 sec, Throughput:3.7903769124687106, Expert Loads saved:0 


In [28]:
result.keys()

odict_keys(['sequences', 'scores', 'past_key_values', 'router_logits'])

## Calculate Perplexity

In [29]:
from datasets import load_dataset

test = load_dataset("allenai/c4", "en", split="validation", cache_dir='~/scratch/', streaming=True)
all_text = ""

for row in list(test.take(1000)):
     all_text += "\n\n" + row['text']

In [30]:
encodings = tokenizer(all_text, return_tensors="pt")


In [31]:
# ! export HF_DATASETS_CACHE="~/scratch/llama3"

In [32]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("/home/hice1/abambhaniya3/scratch/models--meta-llama--Meta-Llama-3-8B-Instruct/LLaMA3-8B-Instrct").to(device)

In [33]:
import torch
from tqdm import tqdm

max_length = 4096
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

 13%|█▎        | 131/1025 [09:55<1:07:41,  4.54s/it]


KeyboardInterrupt: 

In [34]:
ppl

## Mixtral Thresholding 0.25 has 7.7675

tensor(7.7675, device='cuda:0')

In [ ]:
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(nlls, fp)

In [ ]:
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

In [ ]:
"\n\n".join(test["text"])


In [24]:
len(range(0, seq_len, stride)), len(nlls)

(81, 74)